# Training an RL agent with Keras-RL2 using a GEM environment

This notebook serves as an educational introduction to the usage of Keras-RL2 using a GEM environment. Goal of this notebook is to give an understanding what Keras-RL2 is and how to use it to train and evaluate a Reinforcement Learning agent which is able to solve a current control problem of the GEM toolbox.

The following code snippets are only needed if you are executing this file directly from a cloned GitHub repository where you don't have GEM installed

In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent.parent))

## 1. What you need

Before you can start you need to make sure that you have both gym-electric-motor and Keras-rl2 installed. You can install both easily using pip:

- pip install gym-electric-motor
- pip install keras-rl2

Alternatively, you can install them and their latest developer version directly from GitHub:

- https://github.com/upb-lea/gym-electric-motor

You also need to make sure that numpy and gym are installed. You can install both using pip, too. After you have done that you should be able to execute the following cells without any problems.

## 2. Setting up a GEM evironment

This notebook does not focus directly on the usage of GEM and how to set up a GEM environment. If you are new to GEM and interested to find out what it does and how to use it we recommend taking a look at the educational notebook which is dealing with GEM.


For this notebook, we will use a function defined in an external Python file called setting_environment.py. If you are interested to see how we defined our environment's parameters you can take a look at that file. We are using the Discrete Permanent Magnet Synchronous Motor Environment:

- https://upb-lea.github.io/gym-electric-motor/parts/environments/pmsm_disc.html

- https://upb-lea.github.io/gym-electric-motor/parts/physical_systems/electric_motors/pmsm.html

The motor schematic is the following:

![Motor Setup](img/ESBdq1.svg)

And the electrical ODEs for that motor are:

<h3 align="center">

$\frac{\mathrm{d}i_{sq}}{\mathrm{d}t} = \frac{u_{sq}-pL_d\omega_{me}i_{sd}-R_si_{sq}}{L_q}$

$\frac{\mathrm{d}i_{sd}}{\mathrm{d}t} = \frac{u_{sd}-pL_q\omega_{me}i_{sq}-R_si_{sd}}{L_d}$

$\frac{\mathrm{d}\epsilon_{el}}{\mathrm{d}t} = p\omega_{me}$

</h3>

In GEM the agent controls the converter who converts the supply currents to the currents flowing into the motor - in our motor's case $i_{sq}$ and $i_{sd}$:

![Motor Setup](img/SCML_Setting.svg)


In the continuous case the agent's action is controlling the converter's output voltage directly through means of duty cycling. In the discrete case the agent's action decides which converter switches are open and which are closed. Therefore, only a discrete amount of options are available. For this environment we are using the Discrete B6 Bridge Converter which has three switches which amounts to a total of eight possible actions the agent can take:

![Motor Setup](img/B6.svg)



We wish for an agent to be able to control the currents freely. For this a reference generator generates a trajectory which the agent has to follow. Therefore, it has to internally learn to solve the ODEs and convert voltages accordingly. The following code is using our pre-written function set_env to import our pre-defined GEM environment. We are also using the callback RewardLogger to visualize our training progress after training.

In [ ]:
from setting_environment import set_env
env = set_env(training=True)

## 3. Why Keras-RL2?

keras-rl2 is a collection of Reinforcement Learning algorithms implementation. It can be used in a scenario where you want to train an agent of a specific RL algorithm when you don't want to implement the algorithm yourself.

Keras-rl2 is a widely used library for reinforcement learning. Most of the algorithms and their extensions are readily available and are quite stable.However, it is very vaguely implemented. But the ease of adopting to various environments makes it one of the widely used libraries.

For currently available RL algorithms see their documentation:

- https://keras-rl.readthedocs.io/en/latest/agents/overview/

## 4. Training an agent

To train an agent in Keras-rl2 you need two things: The agent and a function approximator representing the control policy. The agent is the algorithm you want to use to solve your problem. The policy defines the function estimation you want to use. Mostly supported are MLP and CNN function estimators. Check the algorithm in the documentation to see what estimator the algorithm you want to use supports. In the future, recurrent functions estimators are supposed to be implemented, too.

### 4.1 Imports

In our control problem we have an environment with a discrete action space. Therefore, we decided for the Deep-Q-Network (DQN):

- https://arxiv.org/abs/1312.5602

For the implementation of the DQN you can check the following github repository:

- https://github.com/keras-rl/keras-rl/blob/master/rl/agents/dqn.py#L89


In this tutorial we use MLP. Necessary Imports like importing DQNAgent from keras-rl2, policy and memory has to be imported
and it is important to Flatten observation, since Keras-rl doesn't support any other form of input. Currently, Feature engineering and invoking the environment are implemeneted in setting_environment.py. Hence it has to be imported too.

In [ ]:
import sys, os
from pathlib import Path
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from setting_environment import set_env

### 4.2 Setting the parameters

For the DQN algorithm we have to define a set of values.These values will be directly used during compilation and training of an Agent

Note: Although we have 8 possible actions, we consider only 7 of them since action 0 and action 7 are same.

In [ ]:
# Training parameters.
gamma = 0.99
time_limit = True
buffer_size = 200000  # number of old obersation steps saved
batch_size = 25  # mini batch size drawn at each update step
env = set_env(time_limit, gamma, training=True)
nb_actions = env.action_space.n
window_length = 1
model = Sequential()
model.add(Flatten(input_shape=(window_length,) + env.observation_space.shape))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

memory = SequentialMemory(limit=200000, window_length=window_length)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(eps=0.2), 'eps', 1, 0.05, 0, 50000)

### 4.3 Starting the training

Once you've setup the environment and defined your parameters starting the training is nothing more than a one-liner. For each algorithm all you have to do is call its .fit() function. The advantage of keras-rl is that you can see all the important parameters and their values during training phase to get an intution of learning. Hence we donot need to implement any callbacks seperately.

In [ ]:
dqn = DQNAgent(
    model=model,
    policy=policy,
    nb_actions=nb_actions,
    memory=memory,
    gamma=gamma,
    batch_size=25,
    train_interval=1,
    memory_interval=1,
    target_model_update=1000,
    nb_steps_warmup=10000,
    enable_double_dqn=True
)

dqn.compile(Adam(lr=1e-4),
            metrics=['mse']
            )

In [ ]:
%matplotlib notebook
dqn.fit(env,
        nb_steps=500000,
        action_repetition=1,
        verbose=2,
        visualize=True,
        nb_max_episode_steps=10000,
        log_interval=1000
        )

### 4.4 Saving the model

When the training is finished, you can save the model that your DQNAgent has learned to reuse it later, e.g. for evaluation or if you want to continue your training. For this, we use "____.save_weights".

In [ ]:
dqn.save_weights('saved_agents\dqn_keras-RL2.hdf5', overwrite=True)

## 5. Evaluating an agent

After you have trained your agent you would like to see how well it does on your control problem. For this you can for example look at a visual representation of your currents in a training phase.
in Testing phase, the widely used keras-rl2 has an edge since we have an option to control which values to be monitored throughout. Keras-rl DQN agent has its own "__.test" method which can be used to evaluate the performance of your agent 

Here the metric used is the total reward agent gets at the end of each episode. 

### 5.1 Loading a model

First, before we start our evaluation we have to load a trained agent or take the trained agent from the same code. To load an trained agent you only have to call the .load_weights(path) function of your algorithm with the respective path. 

If you are directly evaluating the performance(testing), note that the agent has to be compiled first(as shown in some cells above)

In [ ]:
dqn.load_weights('saved_agents\dqn_keras-RL2.hdf5')

In [ ]:
%matplotlib notebook
env = set_env(time_limit = False, gamma=0.99, training=False)
dqn.test(env,
         nb_episodes=5,
         nb_max_episode_steps=100000,
         visualize=True
         )